In [1]:
%pip install pandas
%pip install xlrd
%pip install openpyxl


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

In [4]:
dataset_prod_1 = pd.read_excel("#DATASET/Produtos/Produtos/Produtos_1.xls","Produtos")
dataset_prod_2 = pd.read_excel("#DATASET/Produtos/Produtos/Produtos_2.xls","Query")
dataset_prod_3 = pd.read_excel("#DATASET/Produtos/Produtos/Produtos_3.xls","Query")
dataset_prod_4 = pd.read_excel("#DATASET/Produtos/Produtos/Produtos_4.xls","Query")
dataset_prod_5 = pd.read_excel("#DATASET/Produtos/Produtos/Produtos_5.xls","Query")
dataset_prod_6 = pd.read_excel("#DATASET/Produtos/Produtos/Produtos_6.xls","Query")
dataset_prod_7 = pd.read_excel("#DATASET/Produtos/Produtos/Produtos_7.xls","Query")

In [13]:
dataset_prod_merged = pd.concat([dataset_prod_1,dataset_prod_2,dataset_prod_3,dataset_prod_4,dataset_prod_5,dataset_prod_6,dataset_prod_7])

In [15]:
print(dataset_prod_merged)

      Código                           Nome                      Descrição  \
0          1              TESTE HOMOLOGACAO              TESTE HOMOLOGACAO   
1          2      PENALTY 353026-645 TACTEL      PENALTY 353026-645 TACTEL   
2          3      PENALTY 353028-650 TACTEL      PENALTY 353028-650 TACTEL   
3          4      PENALTY 353028-625 TACTEL      PENALTY 353028-625 TACTEL   
4          5  PENALTY 353025-652 MICROFIBRA  PENALTY 353025-652 MICROFIBRA   
...      ...                            ...                            ...   
1400  396297       COLUMBIA 1464091/464 MHO       COLUMBIA 1464091/464 MHO   
1401  396298       COLUMBIA 1911321/011 PTO       COLUMBIA 1911321/011 PTO   
1402  396299       COLUMBIA 1682201/030 CZA       COLUMBIA 1682201/030 CZA   
1403  396300       COLUMBIA 1682201/010 PTO       COLUMBIA 1682201/010 PTO   
1404  396301        KILDARE 232N MALTE CITY        KILDARE 232N MALTE CITY   

     Unidade  Fornecedor  Marca  Estoque  Situação Tributária  

In [16]:
print(len(dataset_prod_merged))
size_all_prod_datasets = len(dataset_prod_1) + len(dataset_prod_2) + len(dataset_prod_4)+ len(dataset_prod_3)+ len(dataset_prod_5)+ len(dataset_prod_6)+ len(dataset_prod_7)
if len(dataset_prod_merged) == size_all_prod_datasets :
    print("Size is correct!")

if len(dataset_prod_merged.columns) == len(dataset_prod_1.columns):
    print("Number of columns is correct")

print("Concatenation was correctly made!")

394615
Size is correct!
Number of columns is correct
Concatenation was correctly made!


In [18]:
print(dataset_prod_merged.columns)

Index(['Código', 'Nome', 'Descrição', 'Unidade', 'Fornecedor', 'Marca',
       'Estoque', 'Situação Tributária', 'IPI de Entrada', 'IPI de Saída',
       'ICMS de entrada', 'Base de ICMS reduzida na entrada', 'ICMS de saída',
       'Base de ICMS reduzida na saída', 'Referência', 'Localização',
       'Não Incluir na Lista de Preços', 'Margem', 'Classificação Fiscal',
       'Classificação1', 'Classificação2', 'Classificação3', 'Classificação4',
       'Classificação5', 'Classificação6', 'Classificação7', 'Classificação8',
       'Classificação9', 'Observações', 'Código Anterior',
       'Data da última atualização', 'Nome da marca', 'Complemento',
       'Desconto Promocional', 'Saldo', 'Lucratividade', 'Coleção',
       'Comprador', 'Classificação10', 'Classificação11', 'Classificação12',
       'Classificação13', 'Classificação14', 'Classificação15', 'Marcação',
       'CEST', 'EAN', 'NCM', 'Código do Benefício', 'Descrição para Internet',
       'Habilitado para Internet'],
      d

In [19]:
ds_prod = dataset_prod_merged[["Código","Nome","Unidade","Referência","Data da última atualização","Nome da marca", "Complemento", "Classificação11","Classificação12","Classificação13","Classificação14","Classificação15"]]


In [20]:
print(ds_prod.columns)

Index(['Código', 'Nome', 'Unidade', 'Data da última atualização',
       'Nome da marca', 'Classificação11', 'Classificação12',
       'Classificação13', 'Classificação14', 'Classificação15'],
      dtype='object')


In [29]:
ds_cl_prod = pd.read_excel("DATASET/Produtos/Produtos/Classificacoes_dos_produtos.xlsx",nrows=900)

In [32]:
ds_cl_prod_merged = ds_cl_prod[["Classificação","Código","Nome"]]

ds_cl_prod_11 = ds_cl_prod_merged.log[ds_cl_prod_merged['Classificação'] == 11].rename(columns={"Código":"Código_Class_11","Nome":"Nome_Class_11"})

In [33]:
ds_cl_prod_11 = ds_cl_prod_merged.loc[ds_cl_prod_merged['Classificação'] == 11].rename(columns={"Código":"Código_Class_11","Nome":"Nome_Class_11"})
ds_cl_prod_12 = ds_cl_prod_merged.loc[ds_cl_prod_merged['Classificação'] == 12].rename(columns={"Código":"Código_Class_12","Nome":"Nome_Class_12"})
ds_cl_prod_13 = ds_cl_prod_merged.loc[ds_cl_prod_merged['Classificação'] == 13].rename(columns={"Código":"Código_Class_13","Nome":"Nome_Class_13"})
ds_cl_prod_14 = ds_cl_prod_merged.loc[ds_cl_prod_merged['Classificação'] == 14].rename(columns={"Código":"Código_Class_14","Nome":"Nome_Class_14"})
ds_cl_prod_15 = ds_cl_prod_merged.loc[ds_cl_prod_merged['Classificação'] == 15].rename(columns={"Código":"Código_Class_15","Nome":"Nome_Class_15"})


In [34]:
print(ds_cl_prod_15)

     Classificação  Código_Class_15       Nome_Class_15
300             15                0                   .
301             15                1              ABRIGO
302             15                2              AGULHA
303             15                3              AJUSTE
304             15                4             ALICATE
..             ...              ...                 ...
667             15              376            ELASTICO
668             15              377  ESCADA TREINAMENTO
669             15              378         PORTA DOLAR
670             15              379              ESCOVA
671             15              380         TENIS ELBOW

[372 rows x 3 columns]


In [35]:
ds_prod = pd.merge(ds_prod,ds_cl_prod_11[['Código_Class_11','Nome_Class_11']], left_on='Classificação11', right_on='Código_Class_11', how='left').drop(columns=["Código_Class_11"])
ds_prod = pd.merge(ds_prod,ds_cl_prod_12[['Código_Class_12','Nome_Class_12']], left_on='Classificação12', right_on='Código_Class_12', how='left').drop(columns=["Código_Class_12"])
ds_prod = pd.merge(ds_prod,ds_cl_prod_13[['Código_Class_13','Nome_Class_13']], left_on='Classificação13', right_on='Código_Class_13', how='left').drop(columns=["Código_Class_13"])
ds_prod = pd.merge(ds_prod,ds_cl_prod_14[['Código_Class_14','Nome_Class_14']], left_on='Classificação14', right_on='Código_Class_14', how='left').drop(columns=["Código_Class_14"])
ds_prod = pd.merge(ds_prod,ds_cl_prod_15[['Código_Class_15','Nome_Class_15']], left_on='Classificação15', right_on='Código_Class_15', how='left').drop(columns=["Código_Class_15"])

In [36]:
print(ds_prod.tail())

        Código                      Nome Unidade Data da última atualização  \
394610  396297  COLUMBIA 1464091/464 MHO      UN        2022-11-25 00:00:00   
394611  396298  COLUMBIA 1911321/011 PTO      UN        2022-11-25 00:00:00   
394612  396299  COLUMBIA 1682201/030 CZA      UN        2022-11-25 00:00:00   
394613  396300  COLUMBIA 1682201/010 PTO      UN        2022-11-25 00:00:00   
394614  396301   KILDARE 232N MALTE CITY      PR        2022-11-28 00:00:00   

       Nome da marca  Classificação11  Classificação12  Classificação13  \
394610      COLUMBIA                3                2                1   
394611      COLUMBIA                3                2                1   
394612      COLUMBIA                3                2                1   
394613      COLUMBIA                3                2                1   
394614       KILDARE                1                3                2   

        Classificação14  Classificação15 Nome_Class_11 Nome_Class_12  \
39

In [39]:
dataset_detMov_2018_1 = pd.read_excel("DATASET/DetalheMovimentacao/DetalheMovimentacao_2018_Esportes_1.xls","Query")
dataset_detMov_2018_2 = pd.read_excel("DATASET/DetalheMovimentacao/DetalheMovimentacao_2018_Esportes_2.xls","Query")
dataset_detMov_2019_1 = pd.read_excel("DATASET/DetalheMovimentacao/DetalheMovimentacao_2019_Esportes_1.xls","Query")
dataset_detMov_2019_2 = pd.read_excel("DATASET/DetalheMovimentacao/DetalheMovimentacao_2019_Esportes_2.xls","Query")
dataset_detMov_2020_1 = pd.read_excel("DATASET/DetalheMovimentacao/DetalheMovimentacao_2020_Esportes_1.xls","Query")
dataset_detMov_2020_2 = pd.read_excel("DATASET/DetalheMovimentacao/DetalheMovimentacao_2020_Esportes_2.xls","Query")
dataset_detMov_2021_1 = pd.read_excel("DATASET/DetalheMovimentacao/DetalheMovimentacao_2021_Esportes_1.xls","Query")
dataset_detMov_2021_2 = pd.read_excel("DATASET/DetalheMovimentacao/DetalheMovimentacao_2021_Esportes_2.xls","Query")
dataset_detMov_2022 = pd.read_excel("DATASET/DetalheMovimentacao/DetalheMovimentacao_2022_Esportes.xls","Query")


In [40]:
dataset_detMov_merged = pd.concat([dataset_detMov_2018_1,dataset_detMov_2018_2,dataset_detMov_2019_1,dataset_detMov_2019_2,dataset_detMov_2020_1,dataset_detMov_2020_2,dataset_detMov_2021_1,dataset_detMov_2021_2,dataset_detMov_2022])

In [41]:
print(dataset_detMov_merged)

       Empresa  Emitente Série  Número  Item    Data do Movimento  Operação  \
0            6         0   NaN   17710    47  2018-01-03 00:00:00         1   
1            6         0   NaN   17710    48  2018-01-03 00:00:00         1   
2            6         0   NaN   17710    49  2018-01-03 00:00:00         1   
3            6         0   NaN   17710    50  2018-01-03 00:00:00         1   
4            6         0   NaN   17710    51  2018-01-03 00:00:00         1   
...        ...       ...   ...     ...   ...                  ...       ...   
52790        6     26326    NE  358090     2  2022-11-16 00:00:00         4   
52791        6     26326    NE  358090     3  2022-11-16 00:00:00         4   
52792        6     26326    NE  358090     4  2022-11-16 00:00:00         4   
52793        6     26326    NE  358090     5  2022-11-16 00:00:00         4   
52794        6     26326    NE  358090     6  2022-11-16 00:00:00         4   

       Origem ou Destino  Comprador ou Vendedor  Pr

In [42]:
dataset_detMov = dataset_detMov_merged[["Série", "Número", "Item", "Data do Movimento","Operação","Produto","Tamanho","Quantidade","Valor original"]]

In [44]:
ds_tam_aux = pd.read_excel("DATASET/Tamanhos/Tamanhos.xls","Tamanhos")
ds_tam = ds_tam_aux.rename(columns={"Código":"Código_Tamanho","Nome":"Nome_Tamanho"})

In [45]:
dataset_detMov = pd.merge(dataset_detMov,ds_tam, left_on='Tamanho', right_on='Código_Tamanho', how='left').drop(columns=["Código_Tamanho"])

In [46]:
print(dataset_detMov.tail())

          Data do Movimento  Operação  Produto  Tamanho  Quantidade  \
366888  2022-11-16 00:00:00         4   392742      390         2.0   
366889  2022-11-16 00:00:00         4   392742      400         3.0   
366890  2022-11-16 00:00:00         4   392742      410         3.0   
366891  2022-11-16 00:00:00         4   392742      420         2.0   
366892  2022-11-16 00:00:00         4   392742      430         1.0   

        Valor original Nome_Tamanho  
366888       67.274074           39  
366889       67.274074           40  
366890       67.274074           41  
366891       67.274074           42  
366892       67.274074           43  


In [47]:


ds_prod = ds_prod.rename(columns={"Referência":"Referência_Produto", "Complemento":"Complemento_Produto","Código":"Código_Produto","Nome":"Nome_Produto", "Unidade":"Unidade_Produto", "Data da última atualização":"DataUltimaAtualização_Produto","Nome da marca": "NomeMarca_Produto","Classificação11": "Classificação11_Produto","Classificação12": "Classificação12_Produto","Classificação13": "Classificação14_Produto","Classificação14": "Classificação16_Produto","Classificação15": "Classificação15_Produto", "Nome_Class_11": "Nome_Class_11_Produto", "Nome_Class_12": "Nome_Class_12_Produto", "Nome_Class_13": "Nome_Class_13_Produto", "Nome_Class_14": "Nome_Class_14_Produto", "Nome_Class_15": "Nome_Class_15_Produto"})

dataset_detMov = pd.merge(dataset_detMov, ds_prod, left_on='Produto', right_on='Código_Produto', how='left').drop(columns=["Código_Produto"])


In [48]:
print(dataset_detMov.head(5))

     Data do Movimento  Operação  Produto  Tamanho  Quantidade  \
0  2018-01-03 00:00:00         1   265755      130         3.0   
1  2018-01-03 00:00:00         1   340832      130         3.0   
2  2018-01-03 00:00:00         1   237311       95         2.0   
3  2018-01-03 00:00:00         1   237311      100         2.0   
4  2018-01-03 00:00:00         1   237311      105         2.0   

   Valor original Nome_Tamanho                Nome_Produto Unidade_Produto  \
0        3.314815            U  SPEEDO 570003 VME/JOANINHA              UN   
1        3.888889            U            SPEEDO 689032 AZ              UN   
2        7.868519          095           SPEEDO 019019 MHO              UN   
3        7.868519          100           SPEEDO 019019 MHO              UN   
4        7.868519          105           SPEEDO 019019 MHO              UN   

  DataUltimaAtualização_Produto  ... Classificação11_Produto  \
0           2019-10-16 00:00:00  ...                     2.0   
1     

In [56]:
dataset_detMov.to_excel('Dataset_Treated.xlsx', 'Data')